In [1]:
import pandas as pd
import numpy as np
import pydicom 
import os
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras.models import Model, Sequential
from keras.layers import Concatenate, Dense, Input, concatenate, BatchNormalization
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, AveragePooling2D, Flatten

from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


In [2]:
def load_best_slices(path):
    ind = []
    for file in os.listdir(path):
        if file.split('.')[1] == 'txt':
            ind.append(file.split('.')[0])
        
    df = pd.DataFrame(index = ind, columns= ['CT'])
    for ind in df.index:
        df.loc[ind].CT = np.loadtxt(path + ind + '.txt')
    
    return df

df = load_best_slices('best_lung_slice/')

In [3]:
def custom_loss_function(original_array):
    original_array = original_array.astype(np.float32)

    def loss_function(y_true, y_pred):
        
        pred = un_scaler(y_pred, original_array)
        true = un_scaler(y_true, original_array)
        diff = abs(pred-true)
        
        return tf.reduce_mean(diff**4, axis=-1)**.25
    
    return loss_function

def custom_metric_function(original_array):
    original_array = original_array.astype(np.float32)

    def metric_function(y_true, y_pred):
        
        pred = un_scaler(y_pred, original_array)
        true = un_scaler(y_true, original_array)
        diff = abs(pred-true)
        diff = tf.where(diff > 7.5, 7.5, diff)
        diff = tf.where(diff < 0.5, 0.5, diff)
        
        return tf.reduce_mean(diff, axis=-1)
    
    return metric_function

In [4]:
df = df.reset_index()
df = df.rename(columns = {'index' : 'Patient'})

In [5]:
features = pd.read_csv('features.csv')
df = df.merge(features, on= 'Patient')

In [6]:
linear_data_all = pd.read_csv('patient_slope_intercept.csv', index_col=0)
result = pd.DataFrame(index = df.Patient, columns = ['slope'])
    
for ind in result.index:
    result.loc[ind].slope = linear_data_all.loc[ind].slope

df = df.merge(result, on='Patient')
df.iloc[:,2:] = df.iloc[:,2:].astype(np.float32)

In [7]:
#df.info()

In [8]:
dataset = df.values[:,1:]

In [9]:
scaler_features = MinMaxScaler()
dataset[:,1:-1] = scaler_features.fit_transform(dataset[:,1:-1])

In [10]:
def my_scaler(array):
    max_ = array.max()
    min_ = array.min()
    return 0.5*(array-min_)/(max_-min_) + 0.25

def un_scaler(array, original_array):
    max_ = original_array.max()
    min_ = original_array.min()
    return (max_-min_)*2*(array - 0.25) + min_

#y_scaled = my_scaler(result)
# y = un_scaler(y_scaled, result)

In [11]:
scaled_dataset = np.copy(dataset)
scaled_dataset[:,-1] = my_scaler(scaled_dataset[:,-1])

#un_scaler(****, dataset[:,-1])

In [12]:
#just to set the values of the lung matrices between 0 and 1
scaled_dataset[:,0] = scaled_dataset[:,0]/10

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_dataset[:,:-1], scaled_dataset[:,-1], test_size=0.2)

In [14]:
lungs_train = []
for i in range(X_train[:,0].shape[0]):
    lungs_train.append(X_train[:,0][0].astype(np.float32))

lungs_train = np.array(lungs_train)    
lungs_train = lungs_train.reshape(X_train[:,0].shape[0], 512, 512, 1)

lungs_test = []
for i in range(X_test[:,0].shape[0]):
    lungs_test.append(X_test[:,0][0].astype(np.float32))

lungs_test = np.array(lungs_test)    
lungs_test = lungs_test.reshape(X_test[:,0].shape[0], 512, 512, 1)

In [15]:
features_train = []
for i in range(X_train[:,1:].shape[0]):
    features_train.append(X_train[i,1:].astype(np.float32))
    
features_test = []
for i in range(X_test[:,1:].shape[0]):
    features_test.append(X_test[i,1:].astype(np.float32))

In [16]:
features_train = np.array(np.array(features_train))
features_train = features_train.reshape(features_train.shape[0], 8)

features_test = np.array(np.array(features_test))
features_test = features_test.reshape(features_test.shape[0], 8)

In [17]:
inp_conv = Input(shape = (512,512,1), name='lungs')

model_conv = Conv2D(16, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu')(inp_conv)
model_conv = Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu')(model_conv)
model_conv = BatchNormalization()(model_conv)
model_conv = MaxPool2D(pool_size=(3,3))(model_conv)
model_conv = Dropout(0.25)(model_conv)
model_conv = Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu')(model_conv)
model_conv = Conv2D(16, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu')(model_conv)
model_conv = BatchNormalization()(model_conv)
model_conv = MaxPool2D(pool_size=(5,5))(model_conv)
model_conv = Dropout(0.25)(model_conv)
model_conv = Flatten()(model_conv)
model_conv = Dense(16, activation='relu')(model_conv)
outp_conv = Dense(8, activation='sigmoid')(model_conv)

inp_feat = Input(shape = (8,), name='features')
model_feat = Dense(16, activation='relu')(inp_feat)
outp_feat = Dense(8, activation='sigmoid')(model_feat)

model_conc = concatenate([outp_conv, outp_feat])
model_conc = Dense(32, activation='relu')(model_conc)
model_conc = Dense(16, activation='relu')(model_conc)
model_conc = Dense(8, activation='relu')(model_conc)
output = Dense(1, activation='linear')(model_conc)

model = Model(inputs=[inp_conv, inp_feat], outputs=output, name="cnn_nn_model")

In [18]:
model.summary()

Model: "cnn_nn_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
lungs (InputLayer)              (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 510, 510, 16) 160         lungs[0][0]                      
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 508, 508, 32) 4640        conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 508, 508, 32) 128         conv2d_2[0][0]                   
_______________________________________________________________________________________

In [19]:
model.compile(loss=custom_loss_function(dataset[:,-1]), metrics=[custom_metric_function(dataset[:,-1])], optimizer='adam')

In [20]:
model.fit(
    {"lungs": lungs_train, "features": features_train},
    y_train,
    epochs=20, #experimenta mudar este número
    batch_size=14,
    validation_data = ({"lungs": lungs_test, "features": features_test}, y_test)
)

Train on 140 samples, validate on 36 samples
Epoch 1/20
140/140 [==============================] - 93s 664ms/step - loss: 52.1329 - metric_function: 7.4904 - val_loss: 46.1904 - val_metric_function: 7.5000
Epoch 2/20
140/140 [==============================] - 81s 581ms/step - loss: 13.7163 - metric_function: 6.3337 - val_loss: 25.2227 - val_metric_function: 7.4198
Epoch 3/20
140/140 [==============================] - 70s 502ms/step - loss: 6.0779 - metric_function: 4.5380 - val_loss: 24.3568 - val_metric_function: 7.3915
Epoch 4/20
140/140 [==============================] - 64s 457ms/step - loss: 4.6646 - metric_function: 3.8180 - val_loss: 25.3497 - val_metric_function: 7.4438
Epoch 5/20
140/140 [==============================] - 62s 446ms/step - loss: 4.5023 - metric_function: 3.7334 - val_loss: 20.9030 - val_metric_function: 7.2138
Epoch 6/20
140/140 [==============================] - 63s 450ms/step - loss: 4.3839 - metric_function: 3.5258 - val_loss: 20.1753 - val_metric_function: 

In [21]:
un_scaler(y_test.reshape(-1,1), dataset[:,-1])

array([[4.1800713539123535],
       [-2.3159203529357875],
       [-17.04280281066895],
       [7.1971378326415945],
       [7.331258296966546],
       [-0.037748798727985644],
       [0.7165697813034058],
       [-5.953184127807621],
       [0.40556314587592723],
       [-6.741822719573978],
       [-1.1424745321273804],
       [-1.8502906560897827],
       [0.6811119318008423],
       [-9.756552696228027],
       [-2.3561151027679443],
       [-3.5064618587493825],
       [-0.19162534177303314],
       [4.677616596221924],
       [-0.28084152936935425],
       [-4.406274318695068],
       [-2.9042332172393763],
       [-0.568472385406487],
       [-1.4017115831375087],
       [-23.11452865600586],
       [-3.873737335205071],
       [-10.103654861450195],
       [-20.45368194580078],
       [-3.444329261779785],
       [-8.189459800720211],
       [-0.2456692904233968],
       [-5.166130065917965],
       [-4.687772750854492],
       [-1.5106828212738037],
       [-5.131120681762699]

In [22]:
un_scaler(model.predict(([lungs_test, features_test])), dataset[:,-1])

array([[-23.052782],
       [-21.75513 ],
       [-23.162395],
       [-22.99204 ],
       [-20.631706],
       [-22.98773 ],
       [-23.539627],
       [-23.138954],
       [-21.888214],
       [-22.709717],
       [-22.936056],
       [-23.071384],
       [-22.712849],
       [-22.707304],
       [-21.82743 ],
       [-21.83156 ],
       [-23.046206],
       [-23.26302 ],
       [-21.925125],
       [-22.206318],
       [-22.187332],
       [-22.707859],
       [-21.601337],
       [-22.703308],
       [-23.267427],
       [-22.115858],
       [-22.492443],
       [-22.840706],
       [-21.836254],
       [-22.173384],
       [-22.91215 ],
       [-22.358637],
       [-22.27142 ],
       [-23.475264],
       [-22.72893 ],
       [-22.117159]], dtype=float32)

In [23]:
#diz-me que obtemos um número pequenino aqui para eu ficar feliz :D 
print(abs(un_scaler(model.predict(([lungs_test, features_test])), dataset[:,-1]) - un_scaler(y_test.reshape(-1,1), dataset[:,-1])).max())
print(abs(un_scaler(model.predict(([lungs_test, features_test])), dataset[:,-1]) - un_scaler(y_test.reshape(-1,1), dataset[:,-1])).min())
print(abs(un_scaler(model.predict(([lungs_test, features_test])), dataset[:,-1]) - un_scaler(y_test.reshape(-1,1), dataset[:,-1])).mean())

30.189178466796868
0.4112205505371094
18.95711146129502


In [24]:
df[['Patient','slope']]

,Patient,slope
0,ID00329637202285906759848,-1.040238
1,ID00225637202259339837603,-7.460479
2,ID00364637202296074419422,-15.030663
3,ID00130637202220059448013,-8.003377
4,ID00115637202211874187958,-6.403378
...,...,...
171,ID00214637202257820847190,-1.564164
172,ID00032637202181710233084,-14.362868
173,ID00089637202204675567570,-11.805201
174,ID00132637202222178761324,-0.658933


In [25]:
lungs_total = []
for i in range(scaled_dataset[:,0].shape[0]):
    lungs_total.append(scaled_dataset[:,0][0].astype(np.float32))

lungs_total = np.array(lungs_total)    
lungs_total = lungs_total.reshape(scaled_dataset[:,0].shape[0], 512, 512, 1)

In [26]:
new_slope = un_scaler(model.predict([lungs_total, scaled_dataset[:,1:-1]]), dataset[:,-1])

In [27]:
df

,Patient,CT,Percent,Age,FirstWeek,FirstFVC,Height,Male,Ex-smoker,Never smoked,slope
0,ID00329637202285906759848,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",66.249413,69.0,39.0,2805.0,140.940613,1.0,1.0,0.0,-1.040238
1,ID00225637202259339837603,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",80.172195,77.0,13.0,1583.0,113.047203,0.0,0.0,1.0,-7.460479
2,ID00364637202296074419422,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",91.171425,64.0,37.0,3191.0,155.947617,1.0,1.0,0.0,-15.030663
3,ID00130637202220059448013,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",69.852028,65.0,11.0,1690.0,111.074600,0.0,0.0,1.0,-8.003377
4,ID00115637202211874187958,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",77.749298,77.0,15.0,2548.0,134.062927,1.0,1.0,0.0,-6.403378
...,...,...,...,...,...,...,...,...,...,...,...
171,ID00214637202257820847190,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",83.702881,69.0,3.0,2869.0,144.156372,1.0,1.0,0.0,-1.564164
172,ID00032637202181710233084,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",119.629135,63.0,30.0,5045.0,245.212402,1.0,1.0,0.0,-14.362868
173,ID00089637202204675567570,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",60.143166,63.0,9.0,2571.0,124.963547,1.0,0.0,1.0,-11.805201
174,ID00132637202222178761324,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",60.164040,69.0,6.0,2582.0,129.735703,1.0,1.0,0.0,-0.658933


In [28]:
df['pred_slope'] = new_slope

In [29]:
df[['Patient', 'slope','pred_slope']].head(20)

,Patient,slope,pred_slope
0,ID00329637202285906759848,-1.040238,-22.117107
1,ID00225637202259339837603,-7.460479,-21.909973
2,ID00364637202296074419422,-15.030663,-22.070625
3,ID00130637202220059448013,-8.003377,-21.873184
4,ID00115637202211874187958,-6.403378,-22.349203
5,ID00052637202186188008618,-23.287500,-21.428589
6,ID00026637202179561894768,-2.967859,-21.718628
7,ID00183637202241995351650,0.405563,-21.856941
8,ID00229637202260254240583,-3.410314,-23.010849
9,ID00367637202296290303449,-8.287635,-21.796120


In [30]:
print(df.slope.mean())
print(df.pred_slope.mean())

-4.489635944366455
-22.499290466308594
